In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import pandas as pd
import numpy as np
import h5py
import joblib
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline

2025-04-04 10:10:04.084061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743761404.095934  101032 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743761404.099584  101032 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743761404.108884  101032 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743761404.108897  101032 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743761404.108898  101032 computation_placer.cc:177] computation placer alr

# Data Processing



In [2]:
def load_data(filename):
    """
    이미지를 불러옵니다.
    Parameters:
        filename: str
            h5 파일에서 데이터를 불러옵니다.
    Returns:
        np.ndarray, pd.DataFrame, np.ndarray, 
        train 이미지, train spot 정보, test 이미지, test spot 정보
    """
    images, images_test = list(), list()
    spots, spots_test = list(), list()
    with h5py.File(filename, "r") as h5file:
        train_images = h5file["images/Train"]
        train_spots = h5file["spots/Train"]
    
        num_train_slides = len(train_images)
        # Train 이미지를 불러옵니다.
        # 하나의 텐서로 만들기 위해 이미지의 크기를 2000x2000으로 균일하게 만듭니다.
        for i, slide_name in enumerate(train_images.keys()):
            image = np.array(train_images[slide_name])
            p1 = 2000 - image.shape[0]
            p2 = 2000 - image.shape[1]
            images.append(
                np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge')
            )
            spots.append(pd.DataFrame(np.array(train_spots[slide_name])).assign(slide = i))
        # Test 이미지를 불러옵니다.
        test_images = h5file["images/Test"]
        test_spots = h5file["spots/Test"]
        sample = 'S_7'
        image = np.array(test_images[sample])
        p1 = 2000 - image.shape[0]
        p2 = 2000 - image.shape[1]
        images_test.append(np.pad(image, [(0, p1), (0, p2), (0, 0)], 'edge'))
        spots_test.append(pd.DataFrame(np.array(test_spots[sample])).assign(slide = 0))
    # EfficientNet의 형식으로 바꿉니다.
    with tf.device('/CPU:0'):
        images = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images))
    df_spots = pd.concat(spots).reset_index(drop = True)
    with tf.device('/CPU:0'):
        images_test = tf.constant(tf.keras.applications.efficientnet.preprocess_input(images_test))
    df_spots_test = pd.concat(spots_test).reset_index(drop = True)
    return images, df_spots, images_test, df_spots_test

def make_img_proc_info(df, img_with, img_height):
    """
    
    """
    return df.assign(
        left = lambda x: x['x'] - img_width // 2,
        right = lambda x: x['x'] + img_width // 2,
        top = lambda x: x['y'] - img_height // 2,
        bottom = lambda x: x['y'] + img_height // 2,
        lpad = lambda x: -(x['left'].where(x['left'] < 0, 0)),
        rpad = lambda x: -(2000 - x['right']).where(x['right'] > 2000, 0),
        tpad = lambda x: -(x['top'].where(x['top'] < 0, 0)),
        bpad = lambda x: -(2000 - x['bottom']).where(x['bottom'] > 2000, 0)
    ).assign(
        left = lambda x: x['left'].clip(0, 2000),
        right = lambda x: x['right'].clip(0, 2000),
        top = lambda x: x['top'].clip(0, 2000),
        bottom = lambda x: x['bottom'].clip(0, 2000),
    )

def create_tf_ds(df):
    if (pd.Series(targets).isin(df.columns)).all():
        return tf.data.Dataset.from_tensor_slices(
            ({
                i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
            }, df[targets])
        )
    else:
        return tf.data.Dataset.from_tensor_slices({
            i: df[i] for i in ['left', 'right', 'top', 'bottom', 'slide', 'lpad', 'rpad', 'tpad', 'bpad']
        })

def proc_images(X, images):
    return tf.pad(
        images[X['slide'], X['left']:X['right'], X['top']:X['bottom'], :], 
        paddings = [(X['lpad'], X['rpad']), (X['tpad'], X['bpad']), (0, 0)],
        constant_values=1
    )

augmentation_layers = [
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(1.0),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1)
]

def data_augmentation(x):
    for layer in augmentation_layers:
        x = layer(x)
    return x

images, df_spots, images_test, df_spots_test = load_data("data/elucidata_ai_challenge_data.h5")
targets = [i for i in df_spots.columns if i.startswith('C')]

target_proc = make_pipeline(FunctionTransformer(np.log, np.exp),  StandardScaler())
target_proc.fit(df_spots[targets])
df_spots[targets] = target_proc.transform(df_spots[targets])

I0000 00:00:1743761406.345391  101032 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4784 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5


In [3]:
img_width = 224
img_height = 224

df_spots = make_img_proc_info(df_spots, img_width, img_height)
df_spots_test = make_img_proc_info(df_spots_test, img_width, img_height)

df_spots['slide'].unique()

array([0, 1, 2, 3, 4, 5])

In [4]:
class PairwiseHingeLoss(tf.keras.losses.Loss):
    def __init__(self, name="pairwise_hinge_loss"):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        return tf.reduce_mean(
            tf.maximum(
                0.0, 1.0 - (tf.expand_dims(y_pred, axis=-1) - tf.expand_dims(y_pred, axis=-2))
            ) * tf.cast(tf.expand_dims(y_true, axis=-1) > tf.expand_dims(y_true, axis=-2), dtype = tf.float32)
        )

In [5]:
from scipy.stats import spearmanr
def create_model():
    input_shape = (img_width, img_height, 3)
    enet = tf.keras.applications.EfficientNetB0(
        include_top = False, 
        weights = 'imagenet',
        input_shape = input_shape,
        pooling = 'avg'
    )
    inputs = tf.keras.Input(shape = input_shape)
    x = enet(inputs, training = False)
    x = tf.keras.layers.Dropout(0.2)(x)
    d1 = tf.keras.layers.Dense(256, activation = 'relu', kernel_initializer = 'HeUniform')
    x = d1(x)
    d2 = tf.keras.layers.Dense(len(targets))
    outputs = d2(x)
    m = tf.keras.models.Model(inputs, outputs)
    return m, (enet, d1, d2)

def reconstruct_model(layers):
    input_shape = (img_width, img_height, 3)
    inputs = tf.keras.Input(shape = input_shape)
    x = layers[0](inputs, training = True)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = layers[1](x)
    outputs = layers[2](x)
    m = tf.keras.models.Model(inputs, outputs)
    return m

def train_model(
        m, train_idx, valid_idx, learning_rate, 
        target_proc = FunctionTransformer(lambda x: x, lambda x: x), 
        batch_size = 32, epochs = 20, step = ''
    ):
    tf.keras.backend.clear_session()
    ds = create_tf_ds(
        df_spots.iloc[train_idx].pipe(
            lambda x: pd.concat([
                x, x.sample(n = batch_size - (len(x) % batch_size))
            ])
        )
    )
    ds_cv_train = ds.shuffle(5000).map(
        lambda X, Y: (proc_images(X, images), Y)
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

    """
    .map(
        lambda X, Y: (data_augmentation(X), Y)
    )
    """
    ds_cv_prd = ds.map(
        lambda X, Y: (proc_images(X, images), Y)
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
    
    if valid_idx is not None:
        ds_valid = create_tf_ds(df_spots.iloc[valid_idx]).map(
            lambda X, Y: (proc_images(X, images), Y)
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()
    else:
        ds_valid = None
    """
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=learning_rate,
        decay_steps=5000,
        alpha=0.1
    )
    """
    lr_schedule = learning_rate
    m.compile(
        loss = PairwiseHingeLoss(),
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
    )

    df_true_train = df_spots.iloc[train_idx][targets].pipe(
        lambda x: pd.DataFrame(
            target_proc.inverse_transform(x), index = x.index, columns = targets
        )
    )
    if valid_idx is not None:
        df_true = df_spots.iloc[valid_idx][targets].pipe(
            lambda x: pd.DataFrame(
                target_proc.inverse_transform(x), index = x.index, columns = targets
            )
        )
    else:
        df_true = None
    progress_bar = tqdm(total = epochs, desc=step)
    scores_train, scores_valid = list(), list()
    df_prd = None
    for i in range(epochs):
        hist = m.fit(ds_cv_train, epochs = 1, verbose = 0)
        df_prd = pd.DataFrame(
            target_proc.inverse_transform(m.predict(ds_cv_prd, verbose = 0))[:len(df_true_train)], 
            index = df_true_train.index, columns = targets
        )
        scores_train.append(
            df_true_train.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean()
        )
        metric = "train coef: {:.4f}".format(scores_train[-1])
        if valid_idx is not None:
            df_prd = pd.DataFrame(
                target_proc.inverse_transform(m.predict(ds_valid, verbose = 0)), 
                index = df_true.index, columns = targets
            )
            scores_valid.append(
                df_true.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean()
            )
            metric = metric + ", valid coef: {:.4f}".format(scores_valid[-1])
        progress_bar.set_postfix_str(metric)
        progress_bar.update(1)
    progress_bar.close()
    tf.keras.backend.clear_session()
    return scores, df_prd

In [6]:
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(n_splits = 6)

for i, (train_idx, valid_idx) in enumerate(
    gkf.split(df_spots[['x', 'y']], df_spots[targets], groups = df_spots['slide'])
):
    print(df_spots.iloc[train_idx]['slide'].unique(), df_spots.iloc[valid_idx]['slide'].unique())

[0 2 3 4 5] [1]
[1 2 3 4 5] [0]
[0 1 2 3 5] [4]
[0 1 2 4 5] [3]
[0 1 3 4 5] [2]
[0 1 2 3 4] [5]


# Validation

In [ ]:
scores = list()
oofs = list()
for i, (train_idx, valid_idx) in enumerate(
    gkf.split(df_spots[['x', 'y']], df_spots[targets], groups = df_spots['slide'])
):
    m, layers = create_model()
    score_1, df_prd = train_model(
        m, train_idx, valid_idx, learning_rate = 1e-5, 
        batch_size = 32, epochs = 30, step = 'train {}'.format(i + 1)
    )
    """
    m = reconstruct_model(layers)
    score_2, df_prd = train_model(
        m, train_idx, valid_idx, learning_rate = 1e-6, 
        target_proc = target_proc, batch_size = 32, epochs = 10, step = 'fine tuning {}'.format(i + 1)
    )
    """
    scores.append(score_1)
    oofs.append(df_prd)
df_oof = pd.concat(oofs, axis = 0)

# Fine Tuning

과적합을 유의해야하는 데이터셋으로 판단되고 큰 도움은 되지 않습니다.

In [16]:
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = True)
x = tf.keras.layers.Dropout(0.2)(x)
x = d1(x)
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)
m.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),  # Low learning rate
    loss=PairwiseHingeLoss(),
    metrics=[PairwiseHingeLoss()],
)
for i in range(10):
    hist = m.fit(ds_cv_train, epochs=1)
    df_prd =  pd.DataFrame(
       m.predict(ds_valid), index = df_spots[df_spots['slide'] == 5].index, columns = targets
    )
    print(
        df_true.apply(lambda x: spearmanr(x, df_prd.loc[x.name])[0], axis=1).mean(),
        mean_squared_error(df_true.stack(), df_prd.stack())
    )

251/251 ━━━━━━━━━━━━━━━━━━━━ 59s 86ms/step - loss: 11995.6201 - pairwise_hinge_loss: 11995.6201
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 353ms/step
0.6002963380474141 0.8212345117065115
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - loss: 11908.0615 - pairwise_hinge_loss: 11908.0615
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.5968094554895129 0.7898164633285091
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - loss: 11844.4688 - pairwise_hinge_loss: 11844.4688
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.601015406162465 0.8295621284427014
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - loss: 11767.3506 - pairwise_hinge_loss: 11767.3506
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
0.5994918699186993 0.8192455664835954
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - loss: 11700.7139 - pairwise_hinge_loss: 11700.7139
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.603902780624445 0.8455492408696743
251/251 ━━━━━━━━━━━━━━━━━━━━ 22s 87ms/step - loss: 11635.4600 - pairwise_hinge_loss: 11635.4600
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/

# Train

In [8]:
batch_size = 32
ds_train = create_tf_ds(
    df_spots.pipe(
        lambda x: pd.concat([
            x, x.sample(n = batch_size - (len(x) % batch_size))
        ])
    )
).shuffle(5000).map(
    lambda X, Y: (proc_images(X, images), Y)
).map(
    lambda X, Y: (data_augmentation(X), Y)
).batch(batch_size).prefetch(tf.data.AUTOTUNE).cache()

input_shape = (img_width, img_height, 3)
enet = tf.keras.applications.EfficientNetB0(
    include_top = False, 
    weights = 'imagenet',
    input_shape = input_shape,
    pooling = 'avg'
)
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = False)
x = tf.keras.layers.Dropout(0.2)(x)
d1 = tf.keras.layers.Dense(64, activation = 'relu', kernel_initializer = 'HeUniform')
x = d1(x)
d2 = tf.keras.layers.Dense(len(targets))
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=3e-6,
    decay_steps=5000,
    alpha=0.1
)

m.compile(
    loss = PairwiseHingeLoss(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),
    metrics = [PairwiseHingeLoss()]
)
hist = m.fit(ds_train, epochs = 30)

Epoch 1/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 97s 163ms/step - loss: 17552.2695 - pairwise_hinge_loss: 17552.2695
Epoch 2/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 23s 87ms/step - loss: 16380.9219 - pairwise_hinge_loss: 16380.9219
Epoch 3/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 15411.1494 - pairwise_hinge_loss: 15411.1494
Epoch 4/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - loss: 14655.4277 - pairwise_hinge_loss: 14655.4277
Epoch 5/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - loss: 14048.9863 - pairwise_hinge_loss: 14048.9863
Epoch 6/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 23s 86ms/step - loss: 13615.4727 - pairwise_hinge_loss: 13615.4727
Epoch 7/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - loss: 13281.3789 - pairwise_hinge_loss: 13281.3789
Epoch 8/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 12971.7100 - pairwise_hinge_loss: 12971.7100
Epoch 9/30
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 77ms/step - loss: 12757.5244 - pairwise_hinge_loss: 12757.5244
Epoch 10/30
261/261 ━━━━━━━━━━━━━━━━

In [9]:
inputs = tf.keras.Input(shape = input_shape)
x = enet(inputs, training = True)
x = tf.keras.layers.Dropout(0.2)(x)
x = d1(x)
outputs = d2(x)
m = tf.keras.models.Model(inputs, outputs)
m.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),  # Low learning rate
    loss=PairwiseHingeLoss(),
    metrics=[PairwiseHingeLoss()],
)
hist = m.fit(ds_train, epochs=10)

Epoch 1/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 56s 78ms/step - loss: 11561.5879 - pairwise_hinge_loss: 11561.5879
Epoch 2/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 11485.3896 - pairwise_hinge_loss: 11485.3896
Epoch 3/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 11412.0186 - pairwise_hinge_loss: 11412.0186
Epoch 4/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 23s 89ms/step - loss: 11356.1943 - pairwise_hinge_loss: 11356.1943
Epoch 5/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 11279.6406 - pairwise_hinge_loss: 11279.6406
Epoch 6/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 25s 97ms/step - loss: 11218.8232 - pairwise_hinge_loss: 11218.8232
Epoch 7/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 15s 58ms/step - loss: 11159.9199 - pairwise_hinge_loss: 11159.9199
Epoch 8/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 23s 88ms/step - loss: 11097.2285 - pairwise_hinge_loss: 11097.2285
Epoch 9/10
261/261 ━━━━━━━━━━━━━━━━━━━━ 20s 78ms/step - loss: 11019.9414 - pairwise_hinge_loss: 11019.9414
Epoch 10/10
261/261 ━━━━━━━━━━━━━━━━━

In [10]:
joblib.dump(m.get_weights(), 'model/eff_b0_4.joblib')

['model/eff_b0_4.joblib']

In [12]:
ds_test = create_tf_ds(df_spots_test)

df_submission = pd.DataFrame(
    m.predict(
        ds_test.map(lambda X: proc_images(X, images_test)).batch(32)
    ), columns = targets
).reset_index().rename(columns = {'index': 'ID'})

66/66 ━━━━━━━━━━━━━━━━━━━━ 15s 138ms/step


In [ ]:
df_submission